# Matrix Inversion as a QUBO problem
This notebook is a reference implementation for the paper "Floating-Point Calculations on a Quantum Annealer: Division and Matrix Inversion" by M. Rogers and R. Singleton, 2020 [1]. The paper describes an approach to floating-point calculations on a quantum annealer. Specifically, they introduce a binary representation of the floating point variables. This representation is discrete, as they demonstrate their technique for 4 bit and 8 bit accuracy. With this measure they explain and derive the inversion of a matrix as a QUBO problem, which is suitable to run on a DWave quantum annealer. 

[1] Rogers, Michael L., and Robert L. Singleton Jr. "Floating-point calculations on a quantum annealer: Division and matrix inversion." Frontiers in Physics 8 (2020): 265.

In [1]:
import numpy as np
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite
import dwave.inspector

The main challenge in formulating problems for quantum annealers is the description as a binary problem. The authors formulate their problem as a QUBO problem. 

Scalar notation of a QUBO problem: 

\begin{equation} E_{qubo}(a_i, b_{i,j}; q_i) = \sum_{i} a_i q_i + \sum_{i < j} b_{i, j} q_i q_j 
\end{equation} 

Matrix (NxN) inversion is defined as: 

\begin{equation} M \cdot x = y \rightarrow x=M^{-1} \cdot y 
\end{equation} 

Formulate matrix inversion as a quadratic minimization problem with its minimum being the matrix inverse:

\begin{equation} H(x) = (Mx-y)^2 = \sum_{ijk=1}^{N} M_{ki} M_{kj} x^i x^j - 2 \sum_{ij=1}^{N} y_j M_{ji} x^i + \| y \|^2
\end{equation} 


We can obtain a floating point representation of each component of x by expanding in powers of 2 multiplied by boolean-valued variables $q_r^i \in \{0,1\}$ : 


\begin{equation} \chi^i = \sum_{r=0}^{R-1} 2^{-r} q_{r}^{i}
\end{equation} 

\begin{equation} x^i = 2\chi^i -1
\end{equation} 

In [10]:
def compute_chi(q, binary_range):
    # q: Value of binary qubits e.g. q = [0, 1, 0, 1]
    # Binary range is R: amount of binary variables to represent floating precision e.g. 4
    chi = 0
    for r in range(0, binary_range):
        chi += 2**(r) * q[r]
    return chi

def compute_x(chi):
    # This is just an intermediate step...
    return 2*chi - 1

Let's confirm this with the smallest (-1.0) and largest element (2.75) in the range:

In [12]:
# Smallest element
q_start = np.array([0, 0, 0, 0])
# Largest element
q_end = np.array([1, 1, 1, 1])
# Range is length of bidary number
binary_range = q_start.shape[0]
# Compute value...
# x_start = compute_x(compute_chi(q_start, binary_range))
# x_end = compute_x(compute_chi(q_end, binary_range))
x_start = compute_chi(q_start, binary_range)
x_end = compute_chi(q_end, binary_range)
print(x_start)
print(x_end)

0
15


So now our problem can be formulated by expressing x as a function $q_r^i$:

\begin{equation} H[q] = \sum_{i=1}^{N} \sum_{r=0}^{R-1} a_r^i q_r^i + \sum_{i=1}^{N} \sum_{i \neq j}^{N} \sum_{r=0}^{R-1} \sum_{s=0}^{R-1} b_{rs}^{ij} q_r^i q_s^i
\end{equation} 

With this we can formulate our QUBO coefficients. For a detailed derivation please refer to the paper:

\begin{equation} a_r^i = 4 \cdot 2^{-r} \sum_{k} M_{ki} \{ 2^{-r} M_{ki} - (y_k + \sum_{j} M_{kj}) \}
\end{equation} 

\begin{equation} b_{rs}^{ij} = 4 \cdot 2^{-(r+s)} \sum_{k} M_{ki} M_{kj} 
\end{equation} 

In the DWave system QUBO coefficients are addressed with a 1-D index, therefore we have to address the components with a 1-dimensional linear index. So the 2D-indices i (0, N-1) and r (0, R-1) are linarized with the index l. Where l is a usual row-major linear mapping index.

\begin{equation} l(i, r) = i \cdot R + r
\end{equation} 

\begin{equation} M_l = M_{i, r}
\end{equation} 

\begin{equation} i_l = [l / R]
\end{equation} 

\begin{equation} r_l = l  \%  R
\end{equation} 

\begin{equation} a_l = 4 \cdot 2^{-r_l} \sum_{k} M_{ki_l} \{ 2^{-r_l} M_{ki_l} - (y_k + \sum_{j} M_{kj}) \}
\end{equation} 



In [ ]:
def compute_a_l(l, binary_range, M, y, alpha=20):
    # Implementation correct
    # l is the 1D-index
    # binary_range is the amount of digits to represent a floating number
    # M is a matrix of (N x N) 
    # y is a vector
    n = M.shape[0]
    a_l = 0
    # Un-map 1D index
    i_l = l // binary_range
    r_l = l % binary_range
    # Compute ...
    for k in range(n):
        temp_sum = np.sum(M, axis=1)
        a_l += M[k, i_l] * (2**(-r_l) * M[k, i_l] - (y[k] + temp_sum[k]))
    a_l = 4 * 2**(-r_l) * a_l
    return a_l 

\begin{equation} b_{lm} = 4 \cdot 2^{-(r_l+r_{l'})} \sum_{k} M_{ki_l} M_{ki_m} 
\end{equation} 

In [ ]:
def compute_b_l_m(l, m, binary_range, M, alpha=20):
    # Implementation correct
    # r is the position of the digit on the binary number
    # M is a matrix of (N x N) 
    # y is a vector
    n = M.shape[0]
    b_l = 0
    # Un-map 1D index
    i_l = l // binary_range
    i_m = m // binary_range
    r_l = l % binary_range
    r_m = m % binary_range
    # Compute ...
    for k in range(n):
        b_l += M[k, i_l] * M[k, i_m]
    b_l = 4 * (2**(-(r_l+r_m))) * b_l
    # b_l = 4 * (2**(-(r_l+i_m))) * b_l
    return b_l

## Let's imagine a matrix

In [ ]:
# Initialize matrix
M = np.array([[0.5, 1.5], [1.5, 0.5]])
print(M)
# Initialize solution
y = np.array([1.0, 0.0])
print(y)
# Matrix size
n = 2
# Binary bit size
r = 4
# 1-D range
l = n*r

## Construct QUBO problem as BQM for a DWave solver
To implement a usable problem for the DWave solver, we need to construct a matrix Q for our QUBO model. Q contains it's linear coefficient along the diagonal and the quadratic coefficients 

In [ ]:
num_logic_qubits = l
L = {l: compute_a_l(l, binary_range, M, y) for l in range(0, num_logic_qubits)}
Q = {
    (l, m): compute_b_l_m(l, m, binary_range, M)
    for l in range(0, num_logic_qubits)
    for m in range(l, num_logic_qubits)
}
print("Q: ", Q)

In [ ]:
# Sampling solutions for 1000 reads
# sampleset = EmbeddingComposite(DWaveSampler()).sample_qubo(Q, num_reads=1000, label="QUBO Matrix Inversion")
# Transform QUBO into BQM problem
# bqm = dimod.BQM.from_qubo(Q)
offset = np.inner(y, y)
bqm = dimod.BinaryQuadraticModel(L, Q, offset, dimod.Vartype.BINARY)
sampleset = EmbeddingComposite(DWaveSampler()).sample(bqm, num_reads=1000, label="QUBO Matrix Inversion")
dwave.inspector.show(sampleset)
print(sampleset)

In [ ]:
first_solution = np.array(list(sampleset.first.sample.items()))[:, 1]
print(first_solution)

In [ ]:
def compute_x_from_q(q_solution, r, n, i):
    # Implementation should be fine
    # Compute the coefficients of x from solved q: binary -> floating
    x = 0
    for j in range(n*r):
        i_l = j // r
        r_l = j % r
        if i == i_l :
            x += 2**(-r_l) * q_solution[j]
    x = 2 * x - 1
    return x

In [ ]:
x = np.zeros((n))
# Feed through each component of x
for i in range(n):
    x[i] = compute_x_from_q(first_solution, r, n, i)
print(x)

In [ ]:
x = np.zeros((n))
for i in range(n):
    x[i] = compute_x_from_q(np.array([0, 0, 1, 1, 0, 1, 1, 1]), r, n, i)
    # This gives -1: Implementation correct
    # x[i] = compute_x_from_q(np.ones((n*r)), r, n, i)
    # This gives 3: Implementation correct
print(x)

In [ ]:
clasical_result = np.linalg.inv(M).dot(y)
clasical_result